In [ ]:
import numpy as np
import pandas as pd
import fastf1
from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from src.plotset import setup_plot

setup_plot(xyticksize=18,axeslabel=20)

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
# Dictionary to hold strategy -> total points
strategy_points = defaultdict(int)

# Counter for compound usage across all races
compound_counter = Counter()

# Counter for strategy types (1-stop, 2-stop, etc.)
strategy_type_counter = Counter()

# Nested dict: {team: {n-stop: count}}
team_strategy_counts = defaultdict(lambda: defaultdict(int))

In [ ]:
# Loop over all completed races
for rnd in range(1,15):  # adjust if more races are done
    session = fastf1.get_session(2025, rnd, 'R')
    session.load(laps=True, telemetry=False, weather=False)

    # Results table with driver points + status
    results = session.results.set_index('Abbreviation')

    # Go through all drivers
    for drv in session.drivers:
        drv_abbr = session.get_driver(drv)['Abbreviation']
        if rnd==10:
            if drv == '14':
                drv_laps = session.laps.pick_drivers(drv_abbr).iloc[:-4]
            else:
                drv_laps = session.laps.pick_drivers(drv_abbr).iloc[:-3]
        else:
            drv_laps = session.laps.pick_drivers(drv_abbr)
            if rnd==6:
                if drv_laps['Compound'].nunique() == 3:
                    drv_laps['Compound'] = drv_laps['Compound'].apply(lambda x: np.nan if x == 'nan' else x)
                    drv_laps['Stint'].bfill(inplace=True)
                    drv_laps['Compound'].bfill(inplace=True)
                else:
                    drv_laps['Stint'].fillna(0)
                    drv_laps['Compound'] = drv_laps['Compound'].apply(lambda x: 'MEDIUM' if x == 'nan' else x)
            if rnd == 13:
                drv_laps['Compound'] = drv_laps['Compound'].apply(lambda x: 'MEDIUM' if x == 'None' else x)

        if drv_laps.empty:
            continue

        # Skip drivers who retired / did not finish
        if drv_abbr in results.index:
            status = results.loc[drv_abbr, 'Status']
            if isinstance(status, str) and ('Retired' in status or 'DNF' in status or 'Did not start' in status or 'DNS' in status):
                continue
        else:
            continue
        
        # Get team name
        team_name = results.loc[drv_abbr, 'TeamName']

        # Count compounds (for global pie chart)
        compound_counter.update(drv_laps['Compound'])

        # Get stint info -> strategy string (SMH etc.)
        stints = drv_laps[['Stint', 'Compound']].drop_duplicates(subset=['Stint'])
        strategy_str = ''.join(stints['Compound'].str[0])  # e.g. SOFT->S

        # Add to strategy type counter (#stops = len(stints)-1)
        n_stops = len(stints)-1
        strategy_type = f"{n_stops}-stop"
        strategy_type_counter[strategy_type] += 1

        # Increment team counter
        team_strategy_counts[team_name][n_stops] += 1

        # Assign points from results table
        points = results.loc[drv_abbr, 'Points']
        strategy_points[strategy_str] += points

In [ ]:
# ---- Final outputs ----

# Strategy points table
strategy_points_table = pd.DataFrame.from_dict(strategy_points, orient='index', columns=['Points'])
strategy_points_table = strategy_points_table.sort_values(by='Points', ascending=False)

# Compound usage counts
compound_usage = pd.Series(compound_counter).sort_values(ascending=False)

# Strategy type counts
strategy_type_usage = pd.Series(strategy_type_counter).sort_index()

# Convert nested dict to DataFrame
team_strategy_df = pd.DataFrame(team_strategy_counts).fillna(0).astype(int).T
# team_strategy_df = team_strategy_df.sort_index()  # teams in alphabetical order
team_strategy_df.columns = ['5-Stop','1-Stop','2-Stop','3-Stop','4-Stop']
team_strategy_df = team_strategy_df[['1-Stop','2-Stop','3-Stop','4-Stop','5-Stop']]
team_strategy_df['>=3-Stops'] = team_strategy_df['3-Stop'] + team_strategy_df['4-Stop'] + team_strategy_df['5-Stop']
team_strategy_df.drop(['3-Stop','4-Stop','5-Stop'],axis=1,inplace=True)

team_strategy_df['Points'] = [559,194,236,70,52,51,260,20,45,35]
team_strategy_df.sort_values(by='Points',ascending=False,inplace=True)
team_strategy_df.drop('Points',axis=1,inplace=True)

In [ ]:
strategy_points_table

In [ ]:
compound_usage

In [ ]:
strategy_type_usage

In [ ]:
team_strategy_df

In [ ]:
# Your team colors dict (hex or matplotlib-named colors)
colors_dict = {
    'McLaren': '#ff8000',
    'Ferrari': '#e80020',
    'Mercedes': '#27f4d2',
    'Red Bull Racing': '#0600ef',
    'Williams': '#00a0dd',
    'Aston Martin': '#00665f',
    'Kick Sauber': '#00e700',
    'Racing Bulls': '#fcd700',
    'Haas F1 Team': '#b6babd',
    'Alpine': '#ff87bc'
}

# Function to generate lighter shades of a base color
def lighten_color(color, amount=0.5):
    try:
        c = mcolors.cnames[color]
    except:
        c = color
    c = mcolors.to_rgb(c)
    return tuple(1 - (1 - comp) * amount for comp in c)

# Plotting
fig, ax = plt.subplots(figsize=(12,6))

bar_width = 0.2
x = range(len(team_strategy_df))

# Staggered bar positions for 1-stop, 2-stop, >=3
for i, col in enumerate(team_strategy_df.columns):
    for j, team in enumerate(team_strategy_df.index):
        base_color = colors_dict.get(team, 'grey')
        # make lighter shades for 2nd and 3rd bars
        shade = lighten_color(base_color, amount=1 - (i*0.3))  # smaller amount = lighter
        ax.bar(j + i*bar_width, team_strategy_df.loc[team, col], 
               width=bar_width, color=shade, edgecolor='black')

# X-axis labels
# ax.set_xticks([r + bar_width for r in range(len(team_strategy_df))])
ax.set_xticks([])
# ax.set_xticklabels(team_strategy_df.index, rotation=30, ha='right')

ax.set_ylabel("# Times the Strategy was used")
ax.set_title("Team Preferences for 1/2/3+ Stop Strategies")
ax.set_ylim(0,20)
ax.set_yticks(range(0,22,2))
ax.set_yticklabels(['']+[i for i in range(2,22,2)])

# Legend manually constructed (one per stop count, not per team)
legend_patches = [
    plt.Rectangle((0,0),1,1, color="k"),  # placeholder
    plt.Rectangle((0,0),1,1, color="k"),
    plt.Rectangle((0,0),1,1, color="k")
]
legend = ax.legend(
                    [plt.Rectangle((0,0),1,1, color='k', alpha=1),
                    plt.Rectangle((0,0),1,1, color='k', alpha=0.7),
                    plt.Rectangle((0,0),1,1, color='k', alpha=0.4)],
                    ['Bar1: 1-Stop', 'Bar2: 2-Stops', 'Bar3: >=3-Stops'],
                    loc='upper right',facecolor='w'
                )

# Change legend text colors
for text in legend.get_texts():
    text.set_color("k")   # or any hex, e.g. "#FF8000"
    text.set_fontsize(20)     # optional: adjust size
    text.set_fontweight("bold")  # optional: make bold


plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('./media/Reel11/tire_strat.png',dpi=300, bbox_inches='tight')